# Few-shot learning on textual data with siamese neural networks

If you're doing machine learning and meet a classification problem with many categories and few examples per category, it is usually thought that you're in trouble 😨. Acquiring new data to solve this issue is not always easy or even doable. Luckily, we'll see that efficient techniques exist to deal with this situation 🕺. 

This problem of learning with only a few examples per category is called "few-shot learning", and "one-shot learning" in the extreme case of only one example per class (yes, you can even do this and [obtain decent results](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf)!). 

Most of the machine learning research on one-shot learning involves images, but some [recent research papers](https://arxiv.org/abs/1710.10280) address the same problem in the Natural Language Processing (NLP) realm.

In this blog post, I will use siamese neural network to tackle few-shot learning, following a [method that was originally applied to images](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf) and that is nicely explained [here](https://sorenbouma.github.io/blog/oneshot/).

Job title classification provides a good example of a few-shot learning problem in NLP. Say you want to group job titles in different categories or "occupations" (e.g. gather "Programmer" and "Software engineer" in an occupation, and "Sales manager" in another one), then unless you have hundreds of job titles examples per occupation you are facing a few-shot learning problem. The U.S government provides such a job title/occupations taxonomy: the [Standard Occupational Classification](https://www.bls.gov/soc/). I'll use it as a toy dataset understand how few-shot learning with siamese neural networks works.

Let's start by downloading the taxonomy and check what's in there.

In [61]:
from io import StringIO
import requests
import pandas as pd

# Download the Standard Occupation Classification
file_url = 'https://www.onetcenter.org/dl_files/database/db_20_1_text/Sample%20of%20Reported%20Titles.txt'
csv = StringIO(requests.get(file_url).text)

# Load it in a pandas DataFrame and drop a useless column
df = pd.read_csv(csv, sep='\t').drop('Shown in My Next Move', axis=1)

# Remove occupations for which we have only one example, because
# We can't even have a train a test examples for those
occupations_counts = df['O*NET-SOC Code'].value_counts()
multi_examples_occupations = occupations_counts[occupations_counts > 1].index
df = df[df['O*NET-SOC Code'].isin(multi_examples_occupations)]

df['SOC minor group'] = df['O*NET-SOC Code'].apply(lambda x: x[:4])

# Lower all job titles for simplicity
df['Reported Job Title'] = df['Reported Job Title'].str.lower()

df.head(20)

,O*NET-SOC Code,Reported Job Title,SOC minor group
0,11-1011.00,chief diversity officer (cdo),11-1
1,11-1011.00,chief executive officer (ceo),11-1
2,11-1011.00,chief financial officer (cfo),11-1
3,11-1011.00,chief nursing officer,11-1
4,11-1011.00,chief operating officer (coo),11-1
5,11-1011.00,executive director,11-1
6,11-1011.00,executive vice president (evp),11-1
7,11-1011.00,operations vice president,11-1
8,11-1011.00,president,11-1
9,11-1011.00,vice president,11-1


The downloaded file contains job categories codes (first column) and samples of job titles that belong to those categories. To get categories description type the occupation code [here](https://www.onetonline.org/help/online/search). For instance, the first occupation (11-1011.00) is called "Chief Executives" as one can guess from the corresponding job titles examples.

Let's investigate a bit:

In [2]:
df.nunique()  # Count the number of different modalities in each column

O*NET-SOC Code         954
Reported Job Title    7172
dtype: int64

So we have 954 categories ("occupations") for 7172 examples, i.e. 7.5 examples per category on average. We are definitely in the few-shot learning setting. 

Before proceeding with modelling let's create a train and test sets, by putting one example in the test set for each class.

In [63]:
test_set = df.groupby('SOC minor group', as_index=False)['Reported Job Title'].first()
train_set = df[~df['Reported Job Title'].isin(test_set['Reported Job Title'])]

x_train, y_train = train_set['Reported Job Title'], train_set['SOC minor group']
x_test, y_test = test_set['Reported Job Title'], test_set['SOC minor group']

## Building a baseline 🐣

Before experimenting with fancy models, let's establish a strong baseline. We can start by using word embeddings to get a vector representation of each job title, and use a nearest neighbor classifier that is a less likely to overfit than tree-based models or parametric classifiers.

To get the representation of a sentence from pre-trained word embeddings I'll use [Zeugma](https://github.com/nkthiebaut/zeugma), an NLP python library I've written that conveniently provides pre-trained word embeddings in the form of [scikit-learn transformers](http://scikit-learn.org/stable/modules/pipeline.html).

In [4]:
from zeugma import EmbeddingTransformer

# We'll use the GloVe pre-trained embeddings, using the sum of the word embeddings
# of a job title as the embedding vector
embedding = EmbeddingTransformer('glove-twitter-200', aggregation='sum')

Using TensorFlow backend.


In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier

# Our model is a nearest neighbor classifier, the input of which is the sum of the 
# embeddings of words in the job title.
clf = KNeighborsClassifier(n_neighbors=1)
baseline = make_pipeline(embedding, clf)

In [64]:
baseline.fit(x_train, y_train)
print(f'Train accuracy (baseline): {100*baseline.score(x_train, y_train):.2f} %')
print(f'Test accuracy (baseline): {100*baseline.score(x_test, y_test):.2f} %')

Train accuracy (baseline): 84.94 %
Test accuracy (baseline): 31.91 %


Definitely not great but not too bad for a simple baseline model, considering a random guess would have a $\frac{1}{n_{\text{classes}}} \simeq 0.1 \%$ accuracy. 

It may seem hard to beat this simple baseline with a deep learning model due to the high chances of overfitting with such a small dataset, but here come siamese networks to the rescue.


## Few-shot learning with siamese neural networks 👯‍♀️

The nearest neighbor model of the previous section is preforming quite well despite its simplicity, because it uses word embeddings learnt on a [huge NLP dataset from Twitter](https://nlp.stanford.edu/projects/glove/). Using word representations learnt in an unsupervised setting on a bigger dataset is the basic principle of [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning) and reduces the overfit by allowing smaller models to perform well, utltimately giving better performances on the test set.

Although the pre-trained embeddings are valuable, the embedding space used to determine nearest neighbors knows nothing about job titles in particular. There must be a way to learn an embedding space in which jobs belonging to the same occupation category are closer. This is where siamese networks come into play.

The main idea of siamese networks is to learn such a representation by training a model to discriminate between pairs of examples that are in the same category, and pairs of examples that come from different categories. 


### Building the pairs dataset

Let's create positive samples with pairs of job titles corresponding to the same SOC, and negative examples with pairs of job titles sampled from different SOC codes.

In [129]:
import itertools
from random import sample

jobs_left = []
jobs_right = []
target = []

soc_codes = train_set['SOC minor group'].unique()
for code in soc_codes:
    similar_jobs = train_set[train_set['SOC minor group'] == code]['Reported Job Title']
    # Pick 1000 random pairs from the SOC group's job titles combinations 
    group_pairs = list(itertools.combinations(similar_jobs, 2)) 
    positive_pairs = sample(group_pairs, 2000) if len(group_pairs) > 2000 else group_pairs
    jobs_left.extend([p[0] for p in positive_pairs])
    jobs_right.extend([p[1] for p in positive_pairs])
    target.extend([1.]*len(positive_pairs))
    
    other_jobs = df[df['SOC minor group'] != code]['Reported Job Title']
    for i in range(len(positive_pairs)):
        jobs_left.append(np.random.choice(similar_jobs))
        jobs_right.append(np.random.choice(other_jobs))
        target.append(0.)

dataset = pd.DataFrame({
        'job_left': jobs_left,
        'job_right': jobs_right,
        'target': target
    }).sample(frac=1)  # Shuffle dataset

dataset.sample(5)

,job_left,job_right,target
96427,staff genetic counselor,certified professional midwife,1.0
213759,double end tenon operator,fitness instructor,0.0
115334,certified executive chef (cec),cost analyst,0.0
192141,lubricator,trades helper,1.0
182783,repair technician,substation mechanic,1.0


Here we see that the pair "professor" and "spanish instructor" belong to the same job occupation category, while "sports writer" and "animal scientist" are in different occupations. Note that we end up with a much bigger dataset by creating pairs. Indeed the number of the created siamese dataset has 218,669 while the original dataset has only 8,921 samples. Of course we have only artifically increased the dataset size because we have not generated new data, but we'll see that this technique is still very powerful.


### Modelling

The general architecture of the model is based on [this very good tutorial](https://medium.com/mlreview/implementing-malstm-on-kaggles-quora-question-pairs-competition-8b31b0b16a07). I first lighly preprocess the job titles, turn them into index sequences and pad them to get a valide input for the incoming neural network classifier. Once again I use convenient transformers from the [Zeugma](https://github.com/nkthiebaut/zeugma) to do so.

In [130]:
import re
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from zeugma import TextsToSequences, Padder, ItemSelector

max_words_job_title = 10  # To avoid very long job titles we limit them to 10 words
vocab_size = 10000  # Number of most-frequent words kept in the vocabulary

def preprocess_job_titles(job_titles):
    """ Return a list of clean job titles """
    def preprocess_job_title(raw_job_title):
        """ Clean a single job title"""
        job_title = re.sub(r'\(.*\)', '', raw_job_title)  # Remove everything between parenthesis
        return job_title.lower().strip()
    return [preprocess_job_title(jt) for jt in job_titles]
    
pipeline = make_pipeline(
    FunctionTransformer(preprocess_job_titles, validate=False),  # Preprocess the text
    TextsToSequences(num_words=vocab_size),  # Turn word sequences into indexes sequences
    Padder(max_length=max_words_job_title),  # Pad shorter job titles with a dummy index
)

# Note that the preprocessing pipeline must be fit on both the right and left examples
# simultaneously
pipeline.fit(list(dataset['job_left']) + list(dataset['job_right']));

In [131]:
x_left = pipeline.transform(dataset['job_left'])
x_right = pipeline.transform(dataset['job_right'])
x_pairs = [x_left, x_right]   # this will be the input of the siamese network

y_pairs = dataset['target'].values

In [132]:
# We re-use the same embedding as with the baseline model
embedding_layer = embedding.model.get_keras_embedding()

Now that we have created the pairs dataset and preprocessed the job titles we can turn to the siamese model itself. It consists of a cloned sequential network, the input of which is a pair of vectors `x_left` and `x_right`. The last layer of the sequential network for `x_left` is the vector representation of the left job title, and same thing for `x_right` the right input job title. The representations of the right and left inputs are used to compute the similarity between the job titles:

$$\text{sim}\left(x_{l}, x_{r}\right) = \exp \left(-\| f(x_l) - f(x_r) \|_1\right),$$

where $\text{sim} \in [0, 1]$, $\|\cdot\|_1$ is the L1 norm, and $f$ is the function corresponding to the application of the cloned sequential network to the left/right input. 

This setting is called the [Manhattan LSTM](https://www.quora.com/What-is-Manhattan-LSTM) becasue the we'll use LSTMs as the sequential network, and the L1 norm used to compute the distance between two samples of a pair is also called the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry). Here is the corresponding code.

In [133]:
from keras.layers import LSTM, Bidirectional
from keras import Model, Sequential
from keras.layers import Input, Dense, Dropout, Lambda, Subtract
from keras import backend as K

def exponent_neg_manhattan_distance(arms_difference):
    """ Compute the exponent of the opposite of the L1 norm of a vector, to get the left/right inputs
    similarity from the inputs differences. This function is used to turned the unbounded
    L1 distance to a similarity measure between 0 and 1"""
    return K.exp(-K.sum(K.abs(arms_difference), axis=1, keepdims=True))

def siamese_lstm(max_length=max_words_job_title, embedding_layer=embedding_layer):
    """ Define, compile and return a siamese LSTM model """
    input_shape = (max_length,)
    left_input = Input(input_shape, name='left_input')
    right_input = Input(input_shape, name='right_input')

    # Define a single sequential model for both arms.
    # In this example I've chosen a stack of 2 bidirectional LSTMs with a bit of dropout
    seq = Sequential(name='sequential_network')
    seq.add(embedding_layer)
    seq.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    seq.add(Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)))
    
    left_output = seq(left_input)
    right_output = seq(right_input)

    # Here we subtract the neuron values of the last layer from the left arm 
    # with the corresponding values from the right arm
    subtracted = Subtract(name='pair_representations_difference')([left_output, right_output])
    malstm_distance = Lambda(exponent_neg_manhattan_distance, 
                             name='masltsm_distance')(subtracted)

    siamese_net = Model(inputs=[left_input, right_input], outputs=malstm_distance)
    siamese_net.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return siamese_net

siamese_lstm = siamese_lstm()

# Print a summary of the model mainly to know the number of trainable parameters
siamese_lstm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input (InputLayer)         (None, 10)           0                                            
__________________________________________________________________________________________________
right_input (InputLayer)        (None, 10)           0                                            
__________________________________________________________________________________________________
sequential_network (Sequential) (None, 64)           238879696   left_input[0][0]                 
                                                                 right_input[0][0]                
__________________________________________________________________________________________________
pair_representations_difference (None, 64)           0           sequential_network[1][0]         
          

In [135]:
siamese_lstm.fit(x_pairs, y_pairs, validation_split=0.1, epochs=10);

Train on 218669 samples, validate on 24297 samples
Epoch 1/10
218669/218669 [==============================] - 249s 1ms/step - loss: 0.6325 - acc: 0.6406 - val_loss: 0.5472 - val_acc: 0.7231
Epoch 2/10
218669/218669 [==============================] - 239s 1ms/step - loss: 0.5323 - acc: 0.7422 - val_loss: 0.4771 - val_acc: 0.7882
Epoch 3/10
218669/218669 [==============================] - 2723s 12ms/step - loss: 0.4865 - acc: 0.7827 - val_loss: 0.4358 - val_acc: 0.8190
Epoch 4/10
218669/218669 [==============================] - 553s 3ms/step - loss: 0.4578 - acc: 0.8041 - val_loss: 0.4092 - val_acc: 0.8392
Epoch 5/10
218669/218669 [==============================] - 552s 3ms/step - loss: 0.4371 - acc: 0.8202 - val_loss: 0.3863 - val_acc: 0.8522
Epoch 6/10
218669/218669 [==============================] - 436s 2ms/step - loss: 0.4207 - acc: 0.8302 - val_loss: 0.3710 - val_acc: 0.8585
Epoch 7/10
218669/218669 [==============================] - 250s 1ms/step - loss: 0.4073 - acc: 0.8375 - va

Without much effort (light preprocessing, few epochs, no early stopping, no hyper-parameters optimization) we obtain a decent ~87 % accuracy on the validation set. But remember that this is not the final task, here we are only solving the binary classification problem of recognizing pairs of job titles that belong to the same occupations category and pairs of jobs that are sampled from different occupations categories.

To address the initial problem of finding each job title's category we have to compute, for each example in the test set, the similarity score of this example with all the examples in the training set. The predicted category is the one of the closest example in training set. 

In [126]:
x_references = pipeline.transform(x_train)  # Preprocess the training set examples

def get_prediction(job_title):
    """ Get the predicted job title category, and the most similar job title
    in the train set. Note that this way of computing a prediction is highly 
    not optimal, but it'll be sufficient for us now. """
    x = pipeline.transform([job_title])
    # Compute similarities of the job title with all job titles in the train set
    similarities = siamese_lstm.predict([[x[0]]*len(x_references), x_references])
    most_similar_index = np.argmax(similarities)
    # The predicted category is the one of the most similar example from the train set
    prediction = train_set['SOC minor group'].iloc[most_similar_index]
    most_similar_example = train_set['Reported Job Title'].iloc[most_similar_index]
    return prediction, most_similar_example

In [ ]:
sample_idx = 1
pred, most_sim = get_prediction(x_test[sample_idx])
print(f'Sampled test job title: {x_test[sample_idx]}')
print(f'True occupation: {test_set["SOC minor group"].iloc[sample_idx]}')
print(f'Occupation prediction: {pred}')
print(f'Most similar example in train set: {most_sim}') 

In [128]:
from sklearn.metrics import accuracy_score

y_pred = [get_prediction(job_title)[0] for job_title in test_set['Reported Job Title'].iloc[:50]]
print(f'Test accuracy (siamese model): {100*accuracy_score(y_pred, y_test[:50]):.2f} %')

Test accuracy (siamese model): 38.00 %


The siamese model thus outperforms the random guess (accuracy ~0.1 %) and the nearest neighbor baseline (~32 %) by a substantial margin 🤹‍♀️ . Even though it is far from perfect, predicts the right category for a job title 2 times out of 5, while it has to choose between roughly a hundred of them. 

What this means is that the siamese model managed to squeeze some juice out of **all** the examples in the dataset, even accross categories. The original multi-class classification approach does not allow to learn "accross categories" because the categorical cross-entropy that is always used to treat those problems actually treats the multi-class classification tasks as a set of independent binary classification tasks. 

The siamese network approach to the few-shot learning problem is definitely a 
way out with textual data 📚. At the cost of a bit more complex modelling it gives better than standards multi-class classification methods. Give it a shot if you and to classify your data with machine learning but don't have many examples per category, you won't regret it 🎊. 